### Fetch Rewards Coding Exercise - Data Analyst

#### Preprocessing

##### Receipts Data

In [162]:
##Importing libraries
import json
import pandas as pd
import numpy as np
from flatten_json import flatten
from datetime import datetime
from pandas_profiling import ProfileReport

In [171]:
##Load Receipts json file
receipts = pd.read_json('receipts.json', lines= True)
receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [172]:
receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      1119 non-null   object 
 1   bonusPointsEarned        544 non-null    float64
 2   bonusPointsEarnedReason  544 non-null    object 
 3   createDate               1119 non-null   object 
 4   dateScanned              1119 non-null   object 
 5   finishedDate             568 non-null    object 
 6   modifyDate               1119 non-null   object 
 7   pointsAwardedDate        537 non-null    object 
 8   pointsEarned             609 non-null    float64
 9   purchaseDate             671 non-null    object 
 10  purchasedItemCount       635 non-null    float64
 11  rewardsReceiptItemList   679 non-null    object 
 12  rewardsReceiptStatus     1119 non-null   object 
 13  totalSpent               684 non-null    float64
 14  userId                  

In [173]:
receipts.isnull().sum()

_id                          0
bonusPointsEarned          575
bonusPointsEarnedReason    575
createDate                   0
dateScanned                  0
finishedDate               551
modifyDate                   0
pointsAwardedDate          582
pointsEarned               510
purchaseDate               448
purchasedItemCount         484
rewardsReceiptItemList     440
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
dtype: int64

In [183]:
receipts.describe()

,bonusPointsEarned,pointsEarned,purchasedItemCount,totalSpent
count,544.000000,609.000000,635.00000,684.000000
mean,238.893382,585.962890,14.75748,77.796857
std,299.091731,1357.166947,61.13424,347.110349
min,5.000000,0.000000,0.00000,0.000000
25%,5.000000,5.000000,1.00000,1.000000
50%,45.000000,150.000000,2.00000,18.200000
75%,500.000000,750.000000,5.00000,34.960000
max,750.000000,10199.800000,689.00000,4721.950000


In [145]:
###Function to parse the dates
def extract_dates(df, date_fields):
    for i in date_fields:
        df[i] = df[i].astype(str).str.extract('(\d+)').fillna(0)
        df[i] = df[i].apply(lambda time: (datetime.fromtimestamp(int(time)/1000)).strftime("%Y-%m-%d %H:%M:%S"))
    return df

In [146]:
receipts["_id"] = pd.json_normalize(receipts["_id"])
receipt_id = receipts.pop('_id')
receipts.insert(0, 'receipt_id', receipt_id)
date_fields = ['createDate', 'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate', ]
receipts_df = extract_dates(receipts, date_fields)

In [153]:
receipts_df.head()

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 10:25:31,2021-01-03 10:25:31,2021-01-03 10:25:31,2021-01-03 10:25:36,2021-01-03 10:25:31,500.0,2021-01-02 19:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 10:24:43,2021-01-03 10:24:43,2021-01-03 10:24:43,2021-01-03 10:24:48,2021-01-03 10:24:43,150.0,2021-01-02 10:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 10:25:37,2021-01-03 10:25:37,1969-12-31 19:00:00,2021-01-03 10:25:42,1969-12-31 19:00:00,5.0,2021-01-02 19:00:00,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 10:25:34,2021-01-03 10:25:34,2021-01-03 10:25:34,2021-01-03 10:25:39,2021-01-03 10:25:34,5.0,2021-01-02 19:00:00,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 10:25:06,2021-01-03 10:25:06,2021-01-03 10:25:11,2021-01-03 10:25:11,2021-01-03 10:25:06,5.0,2021-01-02 10:25:06,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [155]:
###Create a new dataframe from the RewardReceiptItemList column from receipt data
def extract_reward_receipt(df, col_list):
    df = df.explode('rewardsReceiptItemList')
    df = df[col_list]
    df = pd.json_normalize(json.loads(df.to_json(orient= 'records')))
    df.drop(['rewardsReceiptItemList'], axis=1, inplace= True)
    df.columns= df.columns.str.split('.').str[-1]
    df.insert(0, 'reward_id', df.index +1)
    return df

In [156]:
col_list = ['receipt_id','rewardsReceiptItemList']
rewardsReceiptItemList = extract_reward_receipt(receipts_df, col_list)

In [157]:
rewardsReceiptItemList.head()

,reward_id,receipt_id,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,1,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,5ff1e1bb0a720f052300056b,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,5ff1e1f10a720f052300057a,NaN,NaN,NaN,NaN,False,1,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [175]:
rewardsReceiptItemList.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7381 entries, 0 to 7380
Data columns (total 36 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   reward_id                           7381 non-null   int64  
 1   receipt_id                          7381 non-null   object 
 2   barcode                             3090 non-null   object 
 3   description                         6560 non-null   object 
 4   finalPrice                          6767 non-null   object 
 5   itemPrice                           6767 non-null   object 
 6   needsFetchReview                    813 non-null    object 
 7   partnerItemId                       6941 non-null   object 
 8   preventTargetGapPoints              358 non-null    object 
 9   quantityPurchased                   6767 non-null   float64
 10  userFlaggedBarcode                  337 non-null    object 
 11  userFlaggedNewItem                  323 non

In [185]:
rewardsReceiptItemList.describe()

,reward_id,quantityPurchased,userFlaggedQuantity,originalMetaBriteQuantityPurchased
count,7381.000000,6767.000000,299.000000,15.000000
mean,3691.000000,1.386139,1.872910,1.200000
std,2130.855501,1.204363,1.314823,0.414039
min,1.000000,1.000000,1.000000,1.000000
25%,1846.000000,1.000000,1.000000,1.000000
50%,3691.000000,1.000000,1.000000,1.000000
75%,5536.000000,1.000000,3.000000,1.000000
max,7381.000000,17.000000,5.000000,2.000000


In [176]:
rewardsReceiptItemList.isnull().sum()

reward_id                                0
receipt_id                               0
barcode                               4291
description                            821
finalPrice                             614
itemPrice                              614
needsFetchReview                      6568
partnerItemId                          440
preventTargetGapPoints                7023
quantityPurchased                      614
userFlaggedBarcode                    7044
userFlaggedNewItem                    7058
userFlaggedPrice                      7082
userFlaggedQuantity                   7082
needsFetchReviewReason                7162
pointsNotAwardedReason                7041
pointsPayerId                         6114
rewardsGroup                          5650
rewardsProductPartnerId               5112
userFlaggedDescription                7176
originalMetaBriteBarcode              7310
originalMetaBriteDescription          7371
brandCode                             4781
competitorR

In [160]:
receipts_df.drop(['rewardsReceiptItemList'], axis=1, inplace = True)
receipts_df.head()

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 10:25:31,2021-01-03 10:25:31,2021-01-03 10:25:31,2021-01-03 10:25:36,2021-01-03 10:25:31,500.0,2021-01-02 19:00:00,5.0,FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 10:24:43,2021-01-03 10:24:43,2021-01-03 10:24:43,2021-01-03 10:24:48,2021-01-03 10:24:43,150.0,2021-01-02 10:24:43,2.0,FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 10:25:37,2021-01-03 10:25:37,1969-12-31 19:00:00,2021-01-03 10:25:42,1969-12-31 19:00:00,5.0,2021-01-02 19:00:00,1.0,REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 10:25:34,2021-01-03 10:25:34,2021-01-03 10:25:34,2021-01-03 10:25:39,2021-01-03 10:25:34,5.0,2021-01-02 19:00:00,4.0,FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 10:25:06,2021-01-03 10:25:06,2021-01-03 10:25:11,2021-01-03 10:25:11,2021-01-03 10:25:06,5.0,2021-01-02 10:25:06,2.0,FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [79]:
receipts_df.to_csv('receipts.csv', index=False)

In [152]:
rewardsReceiptItemList.to_csv('rewardsReceiptItemList.csv', index=False)

##### Users Data

In [189]:
users = pd.read_json('users.json', lines= True)
users["_id"] = pd.json_normalize(users["_id"])
user_id = users.pop('_id')
users.insert(0, 'user_id', user_id)
users.head()

,user_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [190]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   user_id       495 non-null    object
 1   active        495 non-null    bool  
 2   createdDate   495 non-null    object
 3   lastLogin     433 non-null    object
 4   role          495 non-null    object
 5   signUpSource  447 non-null    object
 6   state         439 non-null    object
dtypes: bool(1), object(6)
memory usage: 23.8+ KB


In [191]:
users.describe()

,user_id,active,createdDate,lastLogin,role,signUpSource,state
count,495,495,495,433,495,447,439
unique,212,2,212,172,2,2,8
top,54943462e4b07e684157a532,True,{'$date': 1418998882381},{'$date': 1614379156799},consumer,Email,WI
freq,20,494,20,20,413,443,396


In [192]:
users.isnull().sum()

user_id          0
active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
dtype: int64

In [193]:
date_fields = ['createdDate', 'lastLogin']
users_df = extract_dates(users, date_fields)

In [194]:
users_df.head()

,user_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 10:24:04,2021-01-03 10:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 10:24:04,2021-01-03 10:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 10:24:04,2021-01-03 10:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 10:25:30,2021-01-03 10:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 10:24:04,2021-01-03 10:25:37,consumer,Email,WI


In [28]:
users_df = users_df.drop_duplicates('user_id', keep='last')
users_df.to_csv('users.csv', index=False)

#### Brand Data

In [59]:
brands = pd.read_json('brands.json', lines= True)
brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [61]:
brands['topBrand']=brands['topBrand'].apply(lambda x: 'True' if x==1.0 else 'False' if x==0.0 else x)

In [68]:
brands['topBrand'].unique()

array(['False', nan, 'True'], dtype=object)

In [62]:
brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,False,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,False,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,False,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,False,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,False,TEST BRANDCODE @1612366146827


In [63]:
brands["_id"] = pd.json_normalize(brands["_id"])
brands_cpg = pd.json_normalize(brands['cpg'])

In [64]:
brands_cpg.columns

Index(['$ref', '$id.$oid'], dtype='object')

In [65]:
brands['cpg_ref'] = brands_cpg['$ref']
brands['cpg'] = brands_cpg['$id.$oid']

In [66]:
brand_id = brands.pop('_id')
brands.insert(0, 'brand_id', brand_id)

In [67]:
brands.head()

,brand_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode,cpg_ref
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,601ac114be37ce2ead437550,test brand @1612366101024,False,NaN,Cogs
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,5332f5fbe4b03c9a25efd0ba,Starbucks,False,STARBUCKS,Cogs
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,601ac142be37ce2ead437559,test brand @1612366146176,False,TEST BRANDCODE @1612366146176,Cogs
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,601ac142be37ce2ead437559,test brand @1612366146051,False,TEST BRANDCODE @1612366146051,Cogs
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7,test brand @1612366146827,False,TEST BRANDCODE @1612366146827,Cogs


In [69]:
brands.to_csv('brands.csv', index=False)

In [187]:
brands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand_id      1167 non-null   object
 1   barcode       1167 non-null   int64 
 2   category      1012 non-null   object
 3   categoryCode  517 non-null    object
 4   cpg           1167 non-null   object
 5   name          1167 non-null   object
 6   topBrand      555 non-null    object
 7   brandCode     933 non-null    object
 8   cpg_ref       1167 non-null   object
dtypes: int64(1), object(8)
memory usage: 82.2+ KB


In [186]:
brands.isnull().sum()

brand_id          0
barcode           0
category        155
categoryCode    650
cpg               0
name              0
topBrand        612
brandCode       234
cpg_ref           0
dtype: int64